## ChatGPT Stock Prediction

This Notebook follows the methodology in Lopez-Lira and Tang (2024) to generate trading signals from news headlines using the ChatGPT API.

The full paper is available here:

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4412788


In [2]:
import yfinance as yf
import pandas as pd

from openai import OpenAI   # pip install openai>=1.0.0
from openai_key import OPENAI_API_KEY


#### Fetch news items for Apple (AAPL)

In [5]:
ticker = "AAPL"

ticker_yf = yf.Ticker(ticker)

company_name = ticker_yf.info['longName']

raw_news = yf.Search(ticker, news_count=100).news



In [18]:
df = pd.json_normalize(raw_news)
if "providerPublishTime" in df.columns:
    df["datetime"] = pd.to_datetime(df["providerPublishTime"], unit="s")
    df = df.drop(columns="providerPublishTime")
df.head()

,uuid,title,publisher,link,type,relatedTickers,thumbnail.resolutions,datetime
0,58355e46-56e2-3363-8aa3-f69983a6f185,Is Apple Inc. (AAPL) the Most Profitable Blue ...,Insider Monkey,https://finance.yahoo.com/news/apple-inc-aapl-...,STORY,"[AAPL, ^DJI]",[{'url': 'https://s.yimg.com/uu/api/res/1.2/ZW...,2025-04-27 20:07:56
1,d497a316-cea7-38ec-9905-03bbabf6f8da,Watch These Apple Stock Price Levels Ahead of ...,Investopedia,https://finance.yahoo.com/m/d497a316-cea7-38ec...,STORY,[AAPL],[{'url': 'https://s.yimg.com/uu/api/res/1.2/aT...,2025-04-28 02:14:46
2,06c62485-1821-3f4c-9889-a0256585f375,Jim Cramer Says Apple Inc. (AAPL) Just ‘Doesn’...,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-says...,STORY,[AAPL],[{'url': 'https://s.yimg.com/uu/api/res/1.2/fT...,2025-04-27 15:42:20
3,374f4f1a-d37c-3850-9514-15e18a35138f,1 Wall Street Analyst Thinks Apple Is Going to...,Motley Fool,https://finance.yahoo.com/m/374f4f1a-d37c-3850...,STORY,[AAPL],[{'url': 'https://s.yimg.com/uu/api/res/1.2/Vp...,2025-04-27 10:15:00
4,8debdea3-3335-3b66-a3a1-b81231a0d5d4,Bitcoin Dominance Surges To 64%: What It Means...,Benzinga,https://finance.yahoo.com/news/bitcoin-dominan...,STORY,NaN,[{'url': 'https://s.yimg.com/uu/api/res/1.2/.k...,2025-04-28 01:09:33


In [19]:
df = df[['datetime', 'publisher', 'type', 'title']]

print(df.head())

             datetime       publisher   type  \
0 2025-04-27 20:07:56  Insider Monkey  STORY   
1 2025-04-28 02:14:46    Investopedia  STORY   
2 2025-04-27 15:42:20  Insider Monkey  STORY   
3 2025-04-27 10:15:00     Motley Fool  STORY   
4 2025-04-28 01:09:33        Benzinga  STORY   

                                               title  
0  Is Apple Inc. (AAPL) the Most Profitable Blue ...  
1  Watch These Apple Stock Price Levels Ahead of ...  
2  Jim Cramer Says Apple Inc. (AAPL) Just ‘Doesn’...  
3  1 Wall Street Analyst Thinks Apple Is Going to...  
4  Bitcoin Dominance Surges To 64%: What It Means...  


In [20]:
def analyze_headline(headline, company_name):
    """Analyze a headline using the ChatGPT API."""
    prompt = (f"Answer \"YES\" if good news, \"NO\" if bad news, or \"UNKNOWN\" if uncertain in the first line. "
              f"Then elaborate with one short and concise sentence on the next line. "
              f"Is this headline good or bad for the stock price of {company_name} in the short term? \n "
              f"Headline: {headline}"
              )
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            temperature=0,
            messages=[
                {"role": "system",
                 "content": ("Forget all your previous instructions. Pretend you are a financial expert. "
                             "You are a financial expert with stock recommendation experience. ")
                 },
                {"role": "user", "content": prompt}
            ]
        )
        answer = response.choices[0].message.content    # response['choices'][0]['message']['content']
        return answer.strip()
    except Exception as e:
        return f"Error: {e}"

In [21]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [23]:
results = []
output_file = 'news_analysis.csv'

df.head(0).assign(analysis="").to_csv(output_file, index=False)


for index, row in df.iterrows():
  headline = row['title']
  result = analyze_headline(headline, company_name)
  results.append({
      'date_published': row['datetime'],
      'content_type': row['type'],
      'publisher': row['publisher'],
      'title': row['title'],
      'analysis': result
  })

  if len(results) % 10 == 0:
    pd.DataFrame(results).to_csv(output_file, mode='a', header=False, index=False)
    results = []

    print(f"Analysis complete. Results saved to {output_file}.")

Analysis complete. Results saved to news_analysis.csv.
Analysis complete. Results saved to news_analysis.csv.


In [24]:
print(headline, '\n', result)

$5,000 Baby Bonus: Trump's Answer To Falling Birth Rates Or A Political Gimmick? 
 UNKNOWN  
The impact of a baby bonus on Apple's stock price is uncertain as it depends on broader economic factors and consumer behavior.
